In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from binarypredictor.net import FixedSystemNet

In [2]:
R = 8.3143

gf = lambda t: 3815.93 + 109.3029 * t - 1.044523e-20 * t ** 7 - (-7209.5 + 118.2007 * t)
gg = lambda t: -3352 + 215.88 * t - 3.5899325e-21 * t ** 7 - (-15745 + 225.14 * t)

f = lambda x: (1 - x) * gf(t) + x * gg(t) + R * t * ((1 - x) * torch.log(1 - x) + x * torch.log(x)) + (1 - x) * x * (-16402 + 1.14 * t)
g = lambda x: R * t * ((1 - x) * torch.log(1 - x) + x * torch.log(x)) + (1 - x) * x * (-15599)
    
df = lambda x: -16402 - gf(t) + gg(t) + 1.14 * t + 32804 * x - 2.28 * t * x - R * t * torch.log(1 - x) + R * t * torch.log(x)
dg = lambda x: 15599 * (-1 + 2 * x) - R * t * torch.log(1 - x) + R * t * torch.log(x)

In [3]:
n_steps = 100
t_range = torch.arange(1235, 1337, step=1/n_steps).unsqueeze(-1)/1000

In [6]:
train_loader = DataLoader(t_range, batch_size=16, shuffle=True)
net = FixedSystemNet(train=True)

In [7]:
lr = 1e-4

loss_func = nn.MSELoss()
optimizer = Adam(net.parameters(), lr=lr)

for n in range(1000):
    for t in train_loader: 
        out = net(t)
        out = torch.clamp(out, min=1e-10, max=1.-1e-4)  # clamp outputs for numerical stability
        x_1, x_2 = out[:, 0], out[:, 1]
        
        loss_1 = loss_func(f(x_1) - x_1 * df(x_1), g(x_2) - x_2 * dg(x_2))        
        loss_2 = loss_func(f(x_1) - (1 - x_1) * df(x_1), g(x_2) - (1 - x_2) * dg(x_2))

        loss = loss_1 + loss_2 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #print('Loss 1: ', loss_1)
    #print('Loss 2: ', loss_2)
    print('Total loss: ', loss)

Total loss:  tensor(91854664., grad_fn=<AddBackward0>)
Total loss:  tensor(20290898., grad_fn=<AddBackward0>)
Total loss:  tensor(3014635., grad_fn=<AddBackward0>)
Total loss:  tensor(363612.2188, grad_fn=<AddBackward0>)
Total loss:  tensor(52420.9453, grad_fn=<AddBackward0>)
Total loss:  tensor(22510.6680, grad_fn=<AddBackward0>)
Total loss:  tensor(14892.2832, grad_fn=<AddBackward0>)
Total loss:  tensor(18270.3438, grad_fn=<AddBackward0>)
Total loss:  tensor(37413.6250, grad_fn=<AddBackward0>)
Total loss:  tensor(21551.3828, grad_fn=<AddBackward0>)
Total loss:  tensor(4471.2314, grad_fn=<AddBackward0>)
Total loss:  tensor(26946.4863, grad_fn=<AddBackward0>)
Total loss:  tensor(27262.3320, grad_fn=<AddBackward0>)
Total loss:  tensor(10808.0332, grad_fn=<AddBackward0>)
Total loss:  tensor(9268.5898, grad_fn=<AddBackward0>)
Total loss:  tensor(39973.4258, grad_fn=<AddBackward0>)
Total loss:  tensor(22281.8711, grad_fn=<AddBackward0>)
Total loss:  tensor(24415.4316, grad_fn=<AddBackward0

Total loss:  tensor(1.1171, grad_fn=<AddBackward0>)
Total loss:  tensor(0.6864, grad_fn=<AddBackward0>)
Total loss:  tensor(1.0411, grad_fn=<AddBackward0>)
Total loss:  tensor(1.2819, grad_fn=<AddBackward0>)
Total loss:  tensor(1.6747, grad_fn=<AddBackward0>)
Total loss:  tensor(0.8747, grad_fn=<AddBackward0>)
Total loss:  tensor(1.1727, grad_fn=<AddBackward0>)
Total loss:  tensor(3.0496, grad_fn=<AddBackward0>)
Total loss:  tensor(1.2351, grad_fn=<AddBackward0>)
Total loss:  tensor(1.1080, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9380, grad_fn=<AddBackward0>)
Total loss:  tensor(0.7665, grad_fn=<AddBackward0>)
Total loss:  tensor(1.4890, grad_fn=<AddBackward0>)
Total loss:  tensor(1.1856, grad_fn=<AddBackward0>)
Total loss:  tensor(1.3982, grad_fn=<AddBackward0>)
Total loss:  tensor(1.3454, grad_fn=<AddBackward0>)
Total loss:  tensor(1.3940, grad_fn=<AddBackward0>)
Total loss:  tensor(4.5009, grad_fn=<AddBackward0>)
Total loss:  tensor(0.7580, grad_fn=<AddBackward0>)
Total loss: 

Total loss:  tensor(1.8338, grad_fn=<AddBackward0>)
Total loss:  tensor(1.3554, grad_fn=<AddBackward0>)
Total loss:  tensor(1.9537, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9966, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9168, grad_fn=<AddBackward0>)
Total loss:  tensor(0.7315, grad_fn=<AddBackward0>)
Total loss:  tensor(1.1508, grad_fn=<AddBackward0>)
Total loss:  tensor(2.6825, grad_fn=<AddBackward0>)
Total loss:  tensor(1.6521, grad_fn=<AddBackward0>)
Total loss:  tensor(0.8787, grad_fn=<AddBackward0>)
Total loss:  tensor(1.8188, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9767, grad_fn=<AddBackward0>)
Total loss:  tensor(0.5455, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9820, grad_fn=<AddBackward0>)
Total loss:  tensor(0.8744, grad_fn=<AddBackward0>)
Total loss:  tensor(4.9384, grad_fn=<AddBackward0>)
Total loss:  tensor(0.5006, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9014, grad_fn=<AddBackward0>)
Total loss:  tensor(1.0897, grad_fn=<AddBackward0>)
Total loss: 

Total loss:  tensor(1.7597, grad_fn=<AddBackward0>)
Total loss:  tensor(1.3409, grad_fn=<AddBackward0>)
Total loss:  tensor(1.2999, grad_fn=<AddBackward0>)
Total loss:  tensor(1.8855, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9421, grad_fn=<AddBackward0>)
Total loss:  tensor(2.8134, grad_fn=<AddBackward0>)
Total loss:  tensor(2.5865, grad_fn=<AddBackward0>)
Total loss:  tensor(2.1904, grad_fn=<AddBackward0>)
Total loss:  tensor(1.6488, grad_fn=<AddBackward0>)
Total loss:  tensor(1.0140, grad_fn=<AddBackward0>)
Total loss:  tensor(1.2387, grad_fn=<AddBackward0>)
Total loss:  tensor(0.9744, grad_fn=<AddBackward0>)
Total loss:  tensor(1.7485, grad_fn=<AddBackward0>)
Total loss:  tensor(1.2298, grad_fn=<AddBackward0>)
Total loss:  tensor(0.8383, grad_fn=<AddBackward0>)
Total loss:  tensor(0.8875, grad_fn=<AddBackward0>)
Total loss:  tensor(1.6706, grad_fn=<AddBackward0>)
Total loss:  tensor(1.2055, grad_fn=<AddBackward0>)
Total loss:  tensor(1.9985, grad_fn=<AddBackward0>)
Total loss: 

KeyboardInterrupt: 

In [9]:
for t in train_loader: 
    out = net(t)
    print(t, out)

tensor([[1.2386],
        [1.2353],
        [1.2947],
        [1.3266],
        [1.2760],
        [1.2675],
        [1.2415],
        [1.3037],
        [1.3197],
        [1.2904],
        [1.3201],
        [1.2515],
        [1.3367],
        [1.2482],
        [1.3366],
        [1.3164]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.2373],
        [1.2359],
        [1.2962],
        [1.2946],
        [1.2463],
        [1.2510],
        [1.3365],
        [1.2950],
        [1.3244],
        [1.3083],
        [1.2853],
        [1.2883],
        [1.3120],
        [1.3144],
        [1.262

        [1.2450]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.3273],
        [1.3363],
        [1.2631],
        [1.3283],
        [1.3095],
        [1.3126],
        [1.3358],
        [1.3079],
        [1.2621],
        [1.3156],
        [1.3119],
        [1.2753],
        [1.3343],
        [1.3240],
        [1.3349],
        [1.2397]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.83

tensor([[1.2647],
        [1.2432],
        [1.3236],
        [1.3211],
        [1.2939],
        [1.2571],
        [1.2550],
        [1.3008],
        [1.2679],
        [1.2916],
        [1.2935],
        [1.3076],
        [1.2828],
        [1.2718],
        [1.3071],
        [1.2748]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.3303],
        [1.2679],
        [1.2504],
        [1.2690],
        [1.2729],
        [1.3347],
        [1.2896],
        [1.2972],
        [1.2653],
        [1.3160],
        [1.3017],
        [1.3083],
        [1.3102],
        [1.2751],
        [1.280

        [1.3142]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.3057],
        [1.2688],
        [1.2969],
        [1.2357],
        [1.3096],
        [1.2881],
        [1.2370],
        [1.2621],
        [1.2622],
        [1.3312],
        [1.2715],
        [1.2478],
        [1.2947],
        [1.2419],
        [1.2398],
        [1.2411]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.83

        [1.3228]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.2474],
        [1.2480],
        [1.2786],
        [1.3030],
        [1.2698],
        [1.2959],
        [1.2391],
        [1.3002],
        [1.2489],
        [1.2675],
        [1.2993],
        [1.2836],
        [1.2814],
        [1.2531],
        [1.2424],
        [1.2577]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.83

        [1.2367]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.3339],
        [1.2468],
        [1.3050],
        [1.2996],
        [1.2593],
        [1.3257],
        [1.2801],
        [1.3313],
        [1.2509],
        [1.2717],
        [1.2409],
        [1.3086],
        [1.2358],
        [1.3150],
        [1.2945],
        [1.2525]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.83

tensor([[1.2707],
        [1.3086],
        [1.2893],
        [1.2873],
        [1.2818],
        [1.3105],
        [1.2908],
        [1.3060],
        [1.2993],
        [1.3195],
        [1.2530],
        [1.3259],
        [1.3275],
        [1.2778],
        [1.2429],
        [1.2628]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.2573],
        [1.2879],
        [1.3187],
        [1.3143],
        [1.2943],
        [1.2846],
        [1.3230],
        [1.2527],
        [1.3207],
        [1.2955],
        [1.2368],
        [1.2926],
        [1.3129],
        [1.2481],
        [1.335

        [1.3253]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.2627],
        [1.2428],
        [1.2384],
        [1.3096],
        [1.2802],
        [1.2837],
        [1.2732],
        [1.2538],
        [1.2381],
        [1.2575],
        [1.2540],
        [1.3335],
        [1.2815],
        [1.2940],
        [1.3062],
        [1.2672]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.83

        [1.3343]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407]], grad_fn=<SigmoidBackward0>)
tensor([[1.2881],
        [1.2647],
        [1.2779],
        [1.2794],
        [1.2673],
        [1.3032],
        [1.2440],
        [1.3175],
        [1.2573],
        [1.2608],
        [1.2719],
        [1.2583],
        [1.3227],
        [1.3014],
        [1.3111],
        [1.2435]]) tensor([[0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.8303, 0.1407],
        [0.83